In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndi
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
train = np.load('Clean_Data/training_data.npy', allow_pickle=True)
test = np.load('Clean_Data/testing_data.npy', allow_pickle=True)
val = np.load('Clean_Data/validation_data.npy', allow_pickle=True)

In [ ]:
df_train = pd.read_pickle('Clean_Data/training_data.csv')

df_test = pd.read_pickle('Clean_Data/testing_data.csv')

df_val = pd.read_pickle('Clean_Data/validation_data.csv')

In [ ]:
print(df_train.head(3))
print(df_test.head(3))
print(df_val.head(3))

In [ ]:
print(df_train.shape)
print(df_test.shape)
print(df_val.shape)

In [ ]:
_ = plt.imshow(df_train['Image'][0], cmap='gray')
_ = plt.axis('off')
_ = plt.title('Image 1')

In [ ]:
df_train['Image'][0].shape

## Split Data

In [ ]:
X_train = df_train['Image']
y_train = df_train['Label']

X_test = df_test['Image']
y_test = df_test['Label']

X_val = df_val['Image']
y_val = df_val['Label']

In [ ]:
print('Training data shape : ', X_train.shape, y_train.shape)

## Preprocess Data

In [ ]:
data_augmentation = keras.Sequential(
    [layers.experimental.preprocessing.RandomFlip("horizontal"),
     layers.experimental.preprocessing.RandomRotation(0.1),
     layers.experimental.preprocessing.RandomContrast(0.1),
     layers.experimental.preprocessing.RandomCrop(0.1, 0.1),
     layers.experimental.preprocessing.Rescaling(0.1),
     layers.experimental.preprocessing.RandomHeight(0.1), 
     layers.experimental.preprocessing.RandomWidth(0.1)
    ])

In [ ]:
augmented_images = data_augmentation(df_train['Image'][1])
plt.imshow(augmented_images)



In [ ]:
plt.figure(figsize=(10, 10))
for im in range(3):
    augmented_images = data_augmentation(df_train['Image'][im])
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0])
    plt.axis("off")
    


In [ ]:
data_augmentation = keras.Sequential(
    [layers.experimental.preprocessing.RandomFlip("horizontal"),
     layers.experimental.preprocessing.RandomRotation(0.1)
    ])

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in df_train.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
np.load('Clean_Data/validation_data.npy')

In [ ]:
plt.figure(figsize=(10, 10))
for images in X_train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.axis("off")

### Equalize Images

In [ ]:
# Define function to equalize list of images
def im_equalize(df_col, df_new_col):
    im_equalized = []
    for im in range(len(df_col)):
        hist = ndi.histogram(df_col[im], min=0, max=255, bins=256)
        cdf = hist.cumsum() / hist.sum()
        im_equalized.append(cdf[df_col[im]] * 255)
    df_new_col = im_equalized

In [ ]:
im_equalize(df_train['Image'], equal)

In [ ]:
# Verify im_equalize function works
im_equalize(df_train['Image'])

_ = plt.imshow(df_train['Image'][42], cmap='gray')
_ = plt.title('Original')
_ = plt.axis('off')
plt.show()

_ = plt.imshow(df_train['Image_Equalized'][42], cmap='gray')
_ = plt.title('Equalized')
_ = plt.axis('off')
plt.show()

## Model

In [ ]:
model = Sequential()
